In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [37]:
# Read the parquet file
dfs={}

dfs[7] = pd.read_parquet('../data/monthly_merge_data/total/201807_total.parquet')
dfs[8] = pd.read_parquet('../data/monthly_merge_data/total/201808_total.parquet')
dfs[9] = pd.read_parquet('../data/monthly_merge_data/total/201809_total.parquet')
dfs[10] = pd.read_parquet('../data/monthly_merge_data/total/201810_total.parquet')
dfs[11] = pd.read_parquet('../data/monthly_merge_data/total/201811_total.parquet')
dfs[12] = pd.read_parquet('../data/monthly_merge_data/total/201812_total.parquet')

for i in range(7,13):
    print(dfs[i].shape)

(500000, 858)
(500000, 858)
(500000, 858)
(500000, 858)
(500000, 858)
(500000, 858)


In [39]:
# Concatenate the dataframes
df = pd.concat([dfs[7], dfs[8], dfs[9], dfs[10], dfs[11], dfs[12]], axis=0)

MemoryError: Unable to allocate 1.90 GiB for an array with shape (85, 3000000) and data type int64

In [ ]:
print(df.shape)
#dfs를 메모리에서 삭제
del dfs

(3000000, 858)


In [8]:
# Classify columns by data type
dtypes = df.dtypes
numeric_cols = dtypes[dtypes.apply(lambda x: np.issubdtype(x, np.number))].index.tolist()
datetime_cols = dtypes[dtypes == 'datetime64[ns]'].index.tolist()
categorical_cols = dtypes[dtypes == 'category'].index.tolist()
object_cols = dtypes[dtypes == 'object'].index.tolist()

# Print the classification
print("\n=== Column Classification by Data Type ===")
print(f"Number of numeric columns: {len(numeric_cols)}")
print(f"Number of datetime columns: {len(datetime_cols)}")
print(f"Number of categorical columns: {len(categorical_cols)}")
print(f"Number of object columns: {len(object_cols)}")

# Display sample columns of each type (first 5)
if numeric_cols:
    print("\nSample numeric columns:", numeric_cols[:5])
if datetime_cols:
    print("\nSample datetime columns:", datetime_cols[:5])
if categorical_cols:
    print("\nSample categorical columns:", categorical_cols[:5])
if object_cols:
    print("\nSample object columns:", object_cols[:5])


=== Column Classification by Data Type ===
Number of numeric columns: 808
Number of datetime columns: 0
Number of categorical columns: 0
Number of object columns: 50

Sample numeric columns: ['기준년월', '남녀구분코드', '회원여부_이용가능', '회원여부_이용가능_CA', '회원여부_이용가능_카드론']

Sample object columns: ['ID', '연령', 'Segment', '가입통신회사코드', '거주시도명']


In [9]:
object_cols

['ID',
 '연령',
 'Segment',
 '가입통신회사코드',
 '거주시도명',
 '직장시도명',
 '_1순위신용체크구분',
 '_2순위신용체크구분',
 '연회비발생카드수_B0M',
 '상품관련면제카드수_B0M',
 '임직원면제카드수_B0M',
 '우수회원면제카드수_B0M',
 '기타면제카드수_B0M',
 'Life_Stage',
 '자발한도감액횟수_R12M',
 '한도증액횟수_R12M',
 '카드론동의여부',
 'RV전환가능여부',
 '한도심사요청건수',
 '_1순위업종',
 '_2순위업종',
 '_3순위업종',
 '_1순위쇼핑업종',
 '_2순위쇼핑업종',
 '_3순위쇼핑업종',
 '_1순위교통업종',
 '_2순위교통업종',
 '_3순위교통업종',
 '_1순위여유업종',
 '_2순위여유업종',
 '_3순위여유업종',
 '_1순위납부업종',
 '_2순위납부업종',
 '_3순위납부업종',
 '최종카드론_신청경로코드',
 '이용금액대',
 '대표결제방법코드',
 '대표청구지고객주소구분코드',
 '대표청구서수령지구분코드',
 '청구서수령방법',
 '할인건수_R3M',
 '할인건수_B0M',
 '인입횟수_ARS_R6M',
 '이용메뉴건수_ARS_R6M',
 '방문횟수_PC_R6M',
 '방문일수_PC_R6M',
 '방문횟수_앱_R6M',
 'OS구분코드',
 '캠페인접촉건수_R12M',
 '캠페인접촉일수_R12M']

In [3]:
df = pd.read_parquet('../data/monthly_merge_data/total/201807_total.parquet')
df_small = df.iloc[:3000, :]

In [15]:
# 각 데이터프레임의 고유값을 저장할 딕셔너리 생성
unique_values_small = {}
unique_values_full = {}

# 두 데이터프레임의 각 열별 고유값 개수 계산
for col in df.columns[2:]:
    unique_values_small[col] = set(df_small[col].unique())
    unique_values_full[col] = set(df[col].unique())

# 고유값 비교
missing_values = {}
for col in df.columns[2:]:
    missing = unique_values_full[col] - unique_values_small[col]
    if len(missing) > 0:
        missing_values[col] = missing

# 결과 출력
print(f"검사한 전체 열 개수: {len(df.columns)}")
print(f"작은 데이터셋에서 누락된 고유값이 있는 열 개수: {len(missing_values)}")

# 누락된 값이 있는 열에 대한 상세 정보 출력
if missing_values:
    print("\n=== 누락된 고유값이 있는 열 상세 정보 ===")
    counter = 0
    for col, missing in missing_values.items():
        if counter >= 5:  # 가독성을 위해 처음 5개 열만 표시
            break
        print(f"\n열 이름: {col}")
        print(f"전체 데이터셋 고유값 개수: {len(unique_values_full[col])}")
        print(f"작은 데이터셋 고유값 개수: {len(unique_values_small[col])}")
        print(f"작은 데이터셋에서 누락된 고유값 개수: {len(missing)}")
        
        # 누락된 값의 예시 표시 (최대 5개)
        if len(missing) > 0:
            print("누락된 값 예시:", list(missing)[:5])
        counter += 1

검사한 전체 열 개수: 858
작은 데이터셋에서 누락된 고유값이 있는 열 개수: 565

=== 누락된 고유값이 있는 열 상세 정보 ===

열 이름: Segment
전체 데이터셋 고유값 개수: 6
작은 데이터셋 고유값 개수: 4
작은 데이터셋에서 누락된 고유값 개수: 2
누락된 값 예시: [None, 'B']

열 이름: 소지카드수_유효_신용
전체 데이터셋 고유값 개수: 5
작은 데이터셋 고유값 개수: 4
작은 데이터셋에서 누락된 고유값 개수: 1
누락된 값 예시: [4]

열 이름: 입회일자_신용
전체 데이터셋 고유값 개수: 326
작은 데이터셋 고유값 개수: 289
작은 데이터셋에서 누락된 고유값 개수: 37
누락된 값 예시: [19920901, 19940101, 19930501, 20041101, 19910801]

열 이름: 입회경과개월수_신용
전체 데이터셋 고유값 개수: 335
작은 데이터셋 고유값 개수: 289
작은 데이터셋에서 누락된 고유값 개수: 46
누락된 값 예시: [266, 281, 283, 284, 285]

열 이름: 최종탈회후경과월
전체 데이터셋 고유값 개수: 71
작은 데이터셋 고유값 개수: 34
작은 데이터셋에서 누락된 고유값 개수: 37
누락된 값 예시: [1, 2, 3, 4, 5]


## 샘플링 된 데이터로 전처리 시작
segment에 대해서만 다시 고려하면 될 듯 함

In [131]:
# df= df_small
# df_copy = df.copy()

#df_copy에 분리된 복제본 생성
df = df_copy.copy()
df_copy = df.copy()

In [31]:
def check_unique_values(df, column_name):
    # Get unique values and their counts
    value_counts = df[column_name].value_counts()
    
    # Check for NULL values
    null_count = df[column_name].isnull().sum()
    
    print(f"=== Analysis for column: {column_name} ===")
    print("Unique values and their counts:")
    for value, count in value_counts.items():
        print(f"{value}: {count}")
    
    print(f"Total unique values: {len(value_counts)}")
    print(f"NULL values: {null_count}")
    if null_count > 0:
        print(f"NULL percentage: {(null_count/len(df))*100:.2f}%")
    print()

## ID 노터치

## 연령
'20대', '30대', ... -> 20, 30 (str -> int)
'70이상' -> 70

In [32]:
check_unique_values(df, '연령')

=== Analysis for column: 연령 ===
Unique values and their counts:
40대: 960
30대: 751
50대: 651
60대: 300
20대: 244
70대이상: 94
Total unique values: 6
NULL values: 0



In [33]:
import re
# 나이 문자열을 정수로 변환하는 함수
def convert_age(age_str):
    if pd.isna(age_str):
        return None
        
    # '20대', '30대' 등의 형식 처리
    if '대' in age_str:
        # '70대이상'과 같은 케이스도 처리
        if '이상' in age_str:
            return int(re.findall(r'\d+', age_str)[0])
        return int(age_str.replace('대', ''))
    # '70이상' 형식 처리
    elif '이상' in age_str:
        return int(re.findall(r'\d+', age_str)[0])
    # 기타 형식인 경우 숫자만 추출
    else:
        digits = re.findall(r'\d+', age_str)
        if digits:
            return int(digits[0])
        return age_str

# 모든 데이터프레임에 변환 적용
df['연령'] = df['연령'].apply(convert_age)
print(df['연령'].unique())


[40 30 20 60 50 70]


## 가입통신회사코드
s,k,l에 대한 원핫 인코딩 수행
결측치는 000이 들어가게 설계

In [34]:
check_unique_values(df, '가입통신회사코드')

=== Analysis for column: 가입통신회사코드 ===
Unique values and their counts:
S사: 1245
K사: 678
L사: 556
Total unique values: 3
NULL values: 521
NULL percentage: 17.37%



In [35]:
# '가입통신회사코드'에 대한 원핫인코딩 수행
# 접두어 '통신사_'를 사용하여 새로운 열 이름 생성
telecom_dummies = pd.get_dummies(df['가입통신회사코드'], prefix='통신사')

# 원본 열을 삭제하고 더미 변수를 데이터프레임에 연결
df = pd.concat([df.drop('가입통신회사코드', axis=1), telecom_dummies], axis=1)

# 통신사 더미 변수 열에서 결측치를 0으로 채움
telecom_cols = [col for col in df.columns if col.startswith('통신사_')]
df[telecom_cols] = df[telecom_cols].fillna(0)

# 결과 확인을 위해 새로 생성된 통신사 열 출력
print("생성된 통신사 더미 변수 열:", telecom_cols)

생성된 통신사 더미 변수 열: ['통신사_K사', '통신사_L사', '통신사_S사']


In [36]:
check_unique_values(df, '통신사_K사')
check_unique_values(df, '통신사_L사')
check_unique_values(df, '통신사_S사')

=== Analysis for column: 통신사_K사 ===
Unique values and their counts:
False: 2322
True: 678
Total unique values: 2
NULL values: 0

=== Analysis for column: 통신사_L사 ===
Unique values and their counts:
False: 2444
True: 556
Total unique values: 2
NULL values: 0

=== Analysis for column: 통신사_S사 ===
Unique values and their counts:
False: 1755
True: 1245
Total unique values: 2
NULL values: 0



## 거주시도도명, 직장시도명
원핫인코딩 수행
(!) 일치여부에 따른 파생변수가 있어도 괜찮지 않을까 생각해서 만듦

In [17]:
check_unique_values(df, '거주시도명')
check_unique_values(df, '직장시도명')

=== Analysis for column: 거주시도명 ===
Unique values and their counts:
서울: 879
경기: 796
대전: 199
인천: 199
부산: 194
충북: 124
경북: 117
경남: 106
대구: 98
울산: 82
전남: 66
광주: 61
전북: 32
충남: 27
강원: 19
제주: 1
Total unique values: 16
NULL values: 0

=== Analysis for column: 직장시도명 ===
Unique values and their counts:
경기: 733
서울: 730
인천: 176
대전: 169
부산: 165
충북: 118
경북: 109
경남: 107
대구: 94
울산: 73
광주: 60
전남: 59
전북: 48
충남: 26
강원: 25
세종: 9
제주: 3
Total unique values: 17
NULL values: 296
NULL percentage: 9.87%



In [37]:
# '거주시도명'에 대한 원핫인코딩 수행
residence_dummies = pd.get_dummies(df['거주시도명'], prefix='거주')
residence_dummies = residence_dummies.astype(int)  # boolean을 0,1로 변환

# '직장시도명'에 대한 원핫인코딩 수행
workplace_dummies = pd.get_dummies(df['직장시도명'], prefix='직장')
workplace_dummies = workplace_dummies.astype(int)  # boolean을 0,1로 변환

# 원본 열을 삭제하고 더미 변수를 데이터프레임에 연결
df = pd.concat([df.drop(['거주시도명', '직장시도명'], axis=1), 
                residence_dummies, 
                workplace_dummies], axis=1)

# 결과 확인을 위해 새로 생성된 열 출력
residence_cols = [col for col in df.columns if col.startswith('거주_')]
workplace_cols = [col for col in df.columns if col.startswith('직장_')]
print("생성된 거주시도 더미 변수 열:", residence_cols)
print("생성된 직장시도 더미 변수 열:", workplace_cols)

생성된 거주시도 더미 변수 열: ['거주_강원', '거주_경기', '거주_경남', '거주_경북', '거주_광주', '거주_대구', '거주_대전', '거주_부산', '거주_서울', '거주_울산', '거주_인천', '거주_전남', '거주_전북', '거주_제주', '거주_충남', '거주_충북']
생성된 직장시도 더미 변수 열: ['직장_강원', '직장_경기', '직장_경남', '직장_경북', '직장_광주', '직장_대구', '직장_대전', '직장_부산', '직장_서울', '직장_세종', '직장_울산', '직장_인천', '직장_전남', '직장_전북', '직장_제주', '직장_충남', '직장_충북']


In [38]:
# 거주지와 직장 일치 여부를 나타내는 변수 생성
# 지역명에서 접두어 추출
residence_prefixes = [col.replace('거주_', '') for col in residence_cols]
workplace_prefixes = [col.replace('직장_', '') for col in workplace_cols]

# 공통 지역 식별
common_regions = list(set(residence_prefixes) & set(workplace_prefixes))

# 거주지-직장 일치 여부 변수 초기화
df['거주직장일치'] = 0

# 각 지역에 대해 거주지와 직장이 일치하는지 확인
for region in common_regions:
    residence_col = f'거주_{region}'
    workplace_col = f'직장_{region}'
    
    # 해당 지역의 거주지와 직장이 모두 1인 경우 일치로 표시
    matches = (df[residence_col] == 1) & (df[workplace_col] == 1)
    df.loc[matches, '거주직장일치'] = 1

# 결과 확인
print("거주지와 직장 일치 여부:")
print(df['거주직장일치'].value_counts())
print(f"일치 비율: {df['거주직장일치'].mean() * 100:.2f}%")

거주지와 직장 일치 여부:
거주직장일치
1    2589
0     411
Name: count, dtype: int64
일치 비율: 86.30%


##  '_1순위신용체크구분', '_2순위신용체크구분'
원핫인코딩 수행행

In [42]:
check_unique_values(df, '_1순위신용체크구분')
check_unique_values(df, '_2순위신용체크구분')

=== Analysis for column: _1순위신용체크구분 ===
Unique values and their counts:
신용: 2894
체크: 90
Total unique values: 2
NULL values: 16
NULL percentage: 0.53%

=== Analysis for column: _2순위신용체크구분 ===
Unique values and their counts:
신용: 1089
체크: 705
Total unique values: 2
NULL values: 1206
NULL percentage: 40.20%



In [44]:
# 두 열이 동일한 값이 신용으로 있는 행이 있는지 확인
credit_check_match = (df['_1순위신용체크구분'] == df['_2순위신용체크구분'])
credit_check_match_count = credit_check_match.sum()
credit_check_match_rate = credit_check_match.mean()

print(f"두 열이 동일한 값이 있는 행 개수: {credit_check_match_count}")
print(f"두 열이 동일한 값이 있는 행 비율: {credit_check_match_rate * 100:.2f}%")

두 열이 동일한 값이 있는 행 개수: 1032
두 열이 동일한 값이 있는 행 비율: 34.40%


In [45]:
# Get unique combinations of the two columns
combinations = df[['_1순위신용체크구분', '_2순위신용체크구분']].drop_duplicates()

# Count occurrences of each combination
combination_counts = df.groupby(['_1순위신용체크구분', '_2순위신용체크구분']).size().reset_index(name='count')

print("=== Unique combinations and their counts ===")
print(combination_counts.sort_values('count', ascending=False))

# Check if there are any null values
print("\n=== Null value counts ===")
print("_1순위신용체크구분:", df['_1순위신용체크구분'].isnull().sum())
print("_2순위신용체크구분:", df['_2순위신용체크구분'].isnull().sum())

# Calculate percentage of each combination
total = len(df)
combination_counts['percentage'] = (combination_counts['count'] / total * 100).round(2)
print("\n=== Combinations with percentages ===")
print(combination_counts.sort_values('percentage', ascending=False))


=== Unique combinations and their counts ===
  _1순위신용체크구분 _2순위신용체크구분  count
0         신용         신용   1017
1         신용         체크    690
2         체크         신용     72
3         체크         체크     15

=== Null value counts ===
_1순위신용체크구분: 16
_2순위신용체크구분: 1206

=== Combinations with percentages ===
  _1순위신용체크구분 _2순위신용체크구분  count  percentage
0         신용         신용   1017        33.9
1         신용         체크    690        23.0
2         체크         신용     72         2.4
3         체크         체크     15         0.5


In [46]:
# Create boolean one-hot encoding for both columns
credit_check1_dummies = pd.get_dummies(df['_1순위신용체크구분'], prefix='신용체크1')
credit_check2_dummies = pd.get_dummies(df['_2순위신용체크구분'], prefix='신용체크2')

# Convert to boolean (True/False)
credit_check1_dummies = credit_check1_dummies.astype(bool)
credit_check2_dummies = credit_check2_dummies.astype(bool)

# Drop original columns and concatenate the dummy variables
df = pd.concat([df.drop(['_1순위신용체크구분', '_2순위신용체크구분'], axis=1), 
                credit_check1_dummies, 
                credit_check2_dummies], axis=1)

# Print the new column names
credit_check1_cols = [col for col in df.columns if col.startswith('신용체크1_')]
credit_check2_cols = [col for col in df.columns if col.startswith('신용체크2_')]
print("신용체크1 더미 변수:", credit_check1_cols)
print("신용체크2 더미 변수:", credit_check2_cols)

신용체크1 더미 변수: ['신용체크1_신용', '신용체크1_체크']
신용체크2 더미 변수: ['신용체크2_신용', '신용체크2_체크']


##  '연회비발생카드수_B0M', '상품관련면제카드수_B0M', '임직원면제카드수_B0M', '우수회원면제카드수_B0M', '기타면제카드수_B0M',
연회비발생카드수_B0M는 원핫 인코딩 나머지는 제거
사유: 모두 0

In [47]:
check_unique_values(df, '연회비발생카드수_B0M')
check_unique_values(df, '상품관련면제카드수_B0M')
check_unique_values(df, '임직원면제카드수_B0M')
check_unique_values(df, '우수회원면제카드수_B0M')
check_unique_values(df, '기타면제카드수_B0M')


=== Analysis for column: 연회비발생카드수_B0M ===
Unique values and their counts:
0개: 2965
1개이상: 35
Total unique values: 2
NULL values: 0

=== Analysis for column: 상품관련면제카드수_B0M ===
Unique values and their counts:
0개: 3000
Total unique values: 1
NULL values: 0

=== Analysis for column: 임직원면제카드수_B0M ===
Unique values and their counts:
0개: 3000
Total unique values: 1
NULL values: 0

=== Analysis for column: 우수회원면제카드수_B0M ===
Unique values and their counts:
0개: 3000
Total unique values: 1
NULL values: 0

=== Analysis for column: 기타면제카드수_B0M ===
Unique values and their counts:
0개: 3000
Total unique values: 1
NULL values: 0



In [ ]:
#연회비발생카드수_B0M이 '0개'이면 0으로 변환 나머진 1로 변환
df['연회비발생카드수_B0M'] = df['연회비발생카드수_B0M'].apply(lambda x: 0 if x == '0개' else 1)
df['연회비발생카드수_B0M'].value_counts()


KeyError: "[('상품관련면제카드수_B0M', '임직원면제카드수_B0M', '우수회원면제카드수_B0M', '기타면제카드수_B0M')] not found in axis"

In [51]:

df.drop(['상품관련면제카드수_B0M', '임직원면제카드수_B0M', '우수회원면제카드수_B0M', '기타면제카드수_B0M'], axis=1, inplace=True)

In [52]:
check_unique_values(df, '연회비발생카드수_B0M')

=== Analysis for column: 연회비발생카드수_B0M ===
Unique values and their counts:
0: 2965
1: 35
Total unique values: 2
NULL values: 0



In [53]:
## 'Life_Stage'

In [54]:
check_unique_values(df, 'Life_Stage')

=== Analysis for column: Life_Stage ===
Unique values and their counts:
자녀성장(1): 1029
자녀성장(2): 735
자녀출산기: 329
가족구축기: 283
노년생활: 270
자녀독립기: 235
독신: 119
Total unique values: 7
NULL values: 0



In [55]:
# Life_Stage에 대한 원핫 인코딩 수행
# 먼저 고유 값 확인
check_unique_values(df, 'Life_Stage')

# 원핫 인코딩 수행
life_stage_dummies = pd.get_dummies(df['Life_Stage'], prefix='LifeStage')
life_stage_dummies = life_stage_dummies.astype(int)  # boolean을 0,1로 변환

# 원본 열을 삭제하고 더미 변수를 데이터프레임에 연결
df = pd.concat([df.drop('Life_Stage', axis=1), life_stage_dummies], axis=1)

# 결과 확인을 위해 새로 생성된 열 출력
life_stage_cols = [col for col in df.columns if col.startswith('LifeStage_')]
print("생성된 Life Stage 더미 변수 열:", life_stage_cols)
print(f"총 {len(life_stage_cols)}개의 Life Stage 더미 변수가 생성되었습니다.")

=== Analysis for column: Life_Stage ===
Unique values and their counts:
자녀성장(1): 1029
자녀성장(2): 735
자녀출산기: 329
가족구축기: 283
노년생활: 270
자녀독립기: 235
독신: 119
Total unique values: 7
NULL values: 0

생성된 Life Stage 더미 변수 열: ['LifeStage_가족구축기', 'LifeStage_노년생활', 'LifeStage_독신', 'LifeStage_자녀독립기', 'LifeStage_자녀성장(1)', 'LifeStage_자녀성장(2)', 'LifeStage_자녀출산기']
총 7개의 Life Stage 더미 변수가 생성되었습니다.


## '자발한도감액횟수_R12M', '한도증액횟수_R12M'
0회 -> 0
1회 -> 1

In [57]:
check_unique_values(df, '자발한도감액횟수_R12M')
check_unique_values(df, '한도증액횟수_R12M')


=== Analysis for column: 자발한도감액횟수_R12M ===
Unique values and their counts:
0회: 2991
1회: 9
Total unique values: 2
NULL values: 0

=== Analysis for column: 한도증액횟수_R12M ===
Unique values and their counts:
0회: 2667
1회이상: 333
Total unique values: 2
NULL values: 0



In [58]:
df['자발한도감액횟수_R12M'] = df['자발한도감액횟수_R12M'].apply(lambda x: 0 if x == '0회' else 1)
df['한도증액횟수_R12M'] = df['한도증액횟수_R12M'].apply(lambda x: 0 if x == '0회' else 1)
print(df['자발한도감액횟수_R12M'].value_counts())
print(df['한도증액횟수_R12M'].value_counts())


자발한도감액횟수_R12M
0    2991
1       9
Name: count, dtype: int64
한도증액횟수_R12M
0    2667
1     333
Name: count, dtype: int64


## '카드론동의여부', 'RV전환가능여부', '한도심사요청건수
Y -> 0, N -> 1으로 변환
Z -> 0, N -> 1으로 변환
회 제거거

In [60]:
check_unique_values(df, '카드론동의여부')
check_unique_values(df, 'RV전환가능여부')
check_unique_values(df, '한도심사요청건수')

=== Analysis for column: 카드론동의여부 ===
Unique values and their counts:
Y: 2414
N: 586
Total unique values: 2
NULL values: 0

=== Analysis for column: RV전환가능여부 ===
Unique values and their counts:
Z: 2484
N: 516
Total unique values: 2
NULL values: 0

=== Analysis for column: 한도심사요청건수 ===
Unique values and their counts:
0회: 3000
Total unique values: 1
NULL values: 0



In [61]:
df['카드론동의여부'] = df['카드론동의여부'].apply(lambda x: 0 if x == 'Y' else 1)
df['RV전환가능여부'] = df['RV전환가능여부'].apply(lambda x: 0 if x == 'Z' else 1)
df['한도심사요청건수'] = df['한도심사요청건수'].str.replace('회', '').astype(int)

print(df['카드론동의여부'].value_counts())
print(df['RV전환가능여부'].value_counts())
print(df['한도심사요청건수'].value_counts())

카드론동의여부
0    2414
1     586
Name: count, dtype: int64
RV전환가능여부
0    2484
1     516
Name: count, dtype: int64
한도심사요청건수
0    3000
Name: count, dtype: int64


## '_1순위업종', '_2순위업종', '_3순위업종, ...
업종 명으로 열을 만들고 열에 이용금액 열의 값을 활용하여 채워 넣는 방식

In [66]:
check_unique_values(df, '_1순위업종')
check_unique_values(df, '_2순위업종')
check_unique_values(df, '_3순위업종')

=== Analysis for column: _1순위업종 ===
Unique values and their counts:
쇼핑: 1351
납부: 511
교통: 224
사교활동: 165
교육: 99
의료: 44
해외: 38
여유생활: 7
일상생활: 2
Total unique values: 9
NULL values: 559
NULL percentage: 18.63%

=== Analysis for column: _2순위업종 ===
Unique values and their counts:
사교활동: 683
쇼핑: 416
납부: 354
교통: 287
의료: 103
교육: 65
해외: 43
여유생활: 18
일상생활: 8
Total unique values: 9
NULL values: 1023
NULL percentage: 34.10%

=== Analysis for column: _3순위업종 ===
Unique values and their counts:
사교활동: 520
교통: 503
납부: 221
쇼핑: 166
의료: 152
여유생활: 55
해외: 41
일상생활: 27
교육: 17
요식: 16
Total unique values: 10
NULL values: 1282
NULL percentage: 42.73%



In [113]:
# 모든 업종의 고유값 추출
all_industries = pd.concat([
    df['_1순위업종'],
    df['_2순위업종'],
    df['_3순위업종']
]).unique().tolist()
shoping = pd.concat([
    df['_1순위쇼핑업종'],
    df['_2순위쇼핑업종'],
    df['_3순위쇼핑업종']
]).unique().tolist()
traffic = pd.concat([
    df['_1순위교통업종'],
    df['_2순위교통업종'],
    df['_3순위교통업종']
]).unique().tolist()
free = pd.concat([
    df['_1순위여유업종'],
    df['_2순위여유업종'],
    df['_3순위여유업종']
]).unique().tolist()
pay = pd.concat([
    df['_1순위납부업종'],
    df['_2순위납부업종'],
    df['_3순위납부업종']
]).unique().tolist()

print("=== Unique industries ===")
print("전체 업종:", all_industries)
print("쇼핑 업종:", shoping)
print("교통 업종:", traffic)
print("여유 업종:", free)
print("납부 업종:", pay)



=== Unique industries ===
전체 업종: ['납부', '쇼핑', None, '사교활동', '교육', '교통', '해외', '의료', '여유생활', '일상생활', '요식']
쇼핑 업종: ['쇼핑기타', '도소매', '온라인', '마트', None, '슈퍼마켓', '편의점', '백화점', '아울렛']
교통 업종: ['택시', '주유', None, '버스지하철', '정비', '철도버스']
여유 업종: [None, '공연', '여유기타', '운동', 'Pet', '항공', '숙박', '공원']
납부 업종: ['보험료', '통신비', None, '관리비', '납부기타', '가스/전기료']


In [114]:
#각 리스트에 None 값을 제외
all_industries = [industry for industry in all_industries if industry is not None]
shoping = [industry for industry in shoping if industry is not None]
traffic = [industry for industry in traffic if industry is not None]
free = [industry for industry in free if industry is not None]
pay = [industry for industry in pay if industry is not None]

print("=== Unique industries ===")
print("전체 업종:", all_industries)
print("쇼핑 업종:", shoping)
print("교통 업종:", traffic)
print("여유 업종:", free)
print("납부 업종:", pay)


=== Unique industries ===
전체 업종: ['납부', '쇼핑', '사교활동', '교육', '교통', '해외', '의료', '여유생활', '일상생활', '요식']
쇼핑 업종: ['쇼핑기타', '도소매', '온라인', '마트', '슈퍼마켓', '편의점', '백화점', '아울렛']
교통 업종: ['택시', '주유', '버스지하철', '정비', '철도버스']
여유 업종: ['공연', '여유기타', '운동', 'Pet', '항공', '숙박', '공원']
납부 업종: ['보험료', '통신비', '관리비', '납부기타', '가스/전기료']


In [118]:
industries_df = pd.DataFrame(0, index=df.index, columns=all_industries)

# Process industry data
for i in range(1, 4):
    #industry
    industry_col_name = f"_{i}순위업종"
    amount_col_name = f"_{i}순위업종_이용금액"
    mask = df[industry_col_name].notnull()
    
    for idx in df[mask].index:
        industry = df.loc[idx, industry_col_name]
        amount = df.loc[idx, amount_col_name]
        if industry in all_industries:
            industries_df.loc[idx, industry] = amount

print(industries_df.head())

     납부     쇼핑  사교활동  교육    교통  해외  의료  여유생활  일상생활  요식
0  1928   1408   672   0     0   0   0     0     0   0
1  2083   2158     0   0  1659   0   0     0     0   0
2     0  16924  1539   0  1362   0   0     0     0   0
3  2284   2405   774   0     0   0   0     0     0   0
4     0      0     0   0     0   0   0     0     0   0


In [120]:
# Create empty dataframes for each category with 0s
shoping_df = pd.DataFrame(0, index=df.index, columns=shoping)
traffic_df = pd.DataFrame(0, index=df.index, columns=traffic)
free_df = pd.DataFrame(0, index=df.index, columns=free)
pay_df = pd.DataFrame(0, index=df.index, columns=pay)

# Process shoping industry data
for i in range(1, 4):
    # Shoping industry
    industry_col_name = f"_{i}순위쇼핑업종"
    amount_col_name = f"_{i}순위쇼핑업종_이용금액"
    mask = df[industry_col_name].notnull()
    
    for idx in df[mask].index:
        industry = df.loc[idx, industry_col_name]
        amount = df.loc[idx, amount_col_name]
        if industry in shoping:
            shoping_df.loc[idx, industry] = amount
print("=== Processed shoping industry data ===")
print(shoping_df.head())

# Process traffic industry data
for i in range(1, 4):
    industry_col_name = f"_{i}순위교통업종"
    amount_col_name = f"_{i}순위교통업종_이용금액"
    mask = df[industry_col_name].notnull()
    
    for idx in df[mask].index:
        industry = df.loc[idx, industry_col_name]
        amount = df.loc[idx, amount_col_name]
        if industry in traffic:
            traffic_df.loc[idx, industry] = amount

print("=== Processed traffic industry data ===")
print(traffic_df.head())

# Process free (leisure) industry data
for i in range(1, 4):
    industry_col_name = f"_{i}순위여유업종"
    amount_col_name = f"_{i}순위여유업종_이용금액"
    mask = df[industry_col_name].notnull()
    
    for idx in df[mask].index:
        industry = df.loc[idx, industry_col_name]
        amount = df.loc[idx, amount_col_name]
        if industry in free:
            free_df.loc[idx, industry] = amount

print("=== Processed free industry data ===")
print(free_df.head())

# Process payment industry data
for i in range(1, 4):
    industry_col_name = f"_{i}순위납부업종"
    amount_col_name = f"_{i}순위납부업종_이용금액"
    mask = df[industry_col_name].notnull()
    
    for idx in df[mask].index:
        industry = df.loc[idx, industry_col_name]
        amount = df.loc[idx, amount_col_name]
        if industry in pay:
            pay_df.loc[idx, industry] = amount

print("=== Processed payment industry data ===")
print(pay_df.head())

# Combine all industry dataframes with the main dataframe
total_rank_df = pd.concat([industries_df, shoping_df, traffic_df, free_df, pay_df], axis=1)

# Drop the original industry columns that we've now processed
industry_cols_to_drop = []
for i in range(1, 4):
    for category in ['업종', '쇼핑업종', '교통업종', '여유업종', '납부업종']:
        industry_cols_to_drop.append(f"_{i}순위{category}")
        industry_cols_to_drop.append(f"_{i}순위{category}_이용금액")

#df = df.drop(industry_cols_to_drop, axis=1)
print("=== Processed industry data ===")
print(total_rank_df.head())

=== Processed shoping industry data ===
   쇼핑기타   도소매   온라인   마트  슈퍼마켓  편의점  백화점  아울렛
0   242     0     0    0     0    0    0    0
1     0   645     0    0   435  315    0    0
2     0  1038  6094  924     0    0    0    0
3     0     0     0  801   487    0    0    0
4     0     0     0    0     0    0    0    0
=== Processed traffic industry data ===
    택시    주유  버스지하철  정비  철도버스
0  200     0    105   0     0
1    0  1215      0   0   443
2    0  1362      0   0     0
3  208     0      0   0     0
4    0     0      0   0     0
=== Processed free industry data ===
   공연  여유기타  운동  Pet  항공  숙박  공원
0   0     0   0    0   0   0   0
1   0     0   0    0   0   0   0
2   0     0   0    0   0   0   0
3   0     0   0    0   0   0   0
4   0     0   0    0   0   0   0
=== Processed payment industry data ===
    보험료   통신비  관리비  납부기타  가스/전기료
0  1883     0    0    44       0
1     0  2083    0     0       0
2     0     0    0     0       0
3  2284     0    0     0       0
4     0     0    0     0

In [124]:
# df 에서 industry_cols_to_drop을 제거하고, total_rank_df를 추가
df = df.drop(industry_cols_to_drop, axis=1)
df = pd.concat([df, total_rank_df], axis=1)

## '최종카드론_신청경로코드'
1: 콜센터를 통해 신청
2: 지점 방문 신청
7: 인터넷 웹사이트
8: 모바일 앱 (스마트폰)
D: 특정 제휴 채널 또는 디지털 자동화 경로 등 (회사별로 다름)

원핫인코딩 Null값은 모두 0으로 처리

In [128]:
check_unique_values(df, '최종카드론_신청경로코드')


=== Analysis for column: 최종카드론_신청경로코드 ===
Unique values and their counts:
8: 289
7: 159
1: 74
D: 22
2: 9
Total unique values: 5
NULL values: 2447
NULL percentage: 81.57%



In [132]:
# Create dummy variables for 최종카드론_신청경로코드
cardloan_dummies = pd.get_dummies(df['최종카드론_신청경로코드'], prefix='카드론신청경로')

# Drop original column and concat dummy variables
df = pd.concat([df.drop('최종카드론_신청경로코드', axis=1), cardloan_dummies], axis=1)

# Convert to binary (0,1)
cardloan_cols = [col for col in df.columns if col.startswith('카드론신청경로_')]
df[cardloan_cols] = df[cardloan_cols].astype(int)

# Print value counts for verification
for col in cardloan_cols:
    print(f"\n{col} value counts:")
    print(df[col].value_counts())



카드론신청경로_1 value counts:
카드론신청경로_1
0    2926
1      74
Name: count, dtype: int64

카드론신청경로_2 value counts:
카드론신청경로_2
0    2991
1       9
Name: count, dtype: int64

카드론신청경로_7 value counts:
카드론신청경로_7
0    2841
1     159
Name: count, dtype: int64

카드론신청경로_8 value counts:
카드론신청경로_8
0    2711
1     289
Name: count, dtype: int64

카드론신청경로_D value counts:
카드론신청경로_D
0    2978
1      22
Name: count, dtype: int64


## 이용금액대
???
논의 후 처리해야할것 같음

In [134]:
check_unique_values(df, '이용금액대')

=== Analysis for column: 이용금액대 ===
Unique values and their counts:
05.10만원-: 601
01.100만원+: 552
02.50만원+: 543
09.미사용: 484
04.10만원+: 438
03.30만원+: 382
Total unique values: 6
NULL values: 0



## 대표결제방법코드
전부 자동이체
drop

In [136]:
check_unique_values(df, '대표결제방법코드')

=== Analysis for column: 대표결제방법코드 ===
Unique values and their counts:
자동이체: 3000
Total unique values: 1
NULL values: 0



In [137]:
# 대표결제방법코드 drop
df.drop('대표결제방법코드', axis=1, inplace=True)

## 대표청구지고객주소구분코드
원핫인코딩딩

In [142]:
check_unique_values(df, '대표청구서수령지구분코드')
check_unique_values(df, '대표청구지고객주소구분코드')


=== Analysis for column: 대표청구서수령지구분코드 ===
Unique values and their counts:
우편: 1572
이메일: 993
당사페이앱+이메일: 312
K톡명세서+이메일: 70
미수신: 53
Total unique values: 5
NULL values: 0



KeyError: '대표청구지고객주소구분코드'

In [140]:
# Create dummy variables for 대표청구지고객주소구분코드
address_dummies = pd.get_dummies(df['대표청구지고객주소구분코드'], prefix='청구지')

# Drop original column and concatenate dummy variables
df = pd.concat([df.drop('대표청구지고객주소구분코드', axis=1), address_dummies], axis=1)

# Convert to binary (0,1)
address_cols = [col for col in df.columns if col.startswith('청구지_')]
df[address_cols] = df[address_cols].astype(int)

# Print value counts for verification
for col in address_cols:
    print(f"\n{col} value counts:")
    print(df[col].value_counts())


청구지_미확인 value counts:
청구지_미확인
0    1563
1    1437
Name: count, dtype: int64

청구지_주거지 value counts:
청구지_주거지
0    1714
1    1286
Name: count, dtype: int64

청구지_회사 value counts:
청구지_회사
0    2723
1     277
Name: count, dtype: int64


In [143]:
# Create dummy variables for 대표청구서수령지구분코드
receipt_dummies = pd.get_dummies(df['대표청구서수령지구분코드'], prefix='수령지')

# Drop the original column and concatenate the dummy variables
df = pd.concat([df.drop('대표청구서수령지구분코드', axis=1), receipt_dummies], axis=1)

# Convert to binary (0,1)
receipt_cols = [col for col in df.columns if col.startswith('수령지_')]
df[receipt_cols] = df[receipt_cols].astype(int)

# Print value counts for verification
for col in receipt_cols:
    print(f"\n{col} value counts:")
    print(df[col].value_counts())


수령지_K톡명세서+이메일 value counts:
수령지_K톡명세서+이메일
0    2930
1      70
Name: count, dtype: int64

수령지_당사페이앱+이메일 value counts:
수령지_당사페이앱+이메일
0    2688
1     312
Name: count, dtype: int64

수령지_미수신 value counts:
수령지_미수신
0    2947
1      53
Name: count, dtype: int64

수령지_우편 value counts:
수령지_우편
1    1572
0    1428
Name: count, dtype: int64

수령지_이메일 value counts:
수령지_이메일
0    2007
1     993
Name: count, dtype: int64


## 청구서수령방법
원핫인코딩딩

In [146]:
check_unique_values(df, '청구서수령방법')

=== Analysis for column: 청구서수령방법 ===
Unique values and their counts:
우편: 1572
이메일: 993
문자메세지: 312
K톡: 70
미수령: 53
Total unique values: 5
NULL values: 0



In [147]:
# Create dummy variables for 청구서수령방법
bill_receipt_dummies = pd.get_dummies(df['청구서수령방법'], prefix='청구지수령')

# Drop original column and concatenate dummy variables
df = pd.concat([df.drop('청구서수령방법', axis=1), bill_receipt_dummies], axis=1)

# Convert to binary (0,1)
bill_receipt_cols = [col for col in df.columns if col.startswith('청구지수령_')]
df[bill_receipt_cols] = df[bill_receipt_cols].astype(int)

# Print value counts for verification
for col in bill_receipt_cols:
    print(f"\n{col} value counts:")
    print(df[col].value_counts())


청구지수령_K톡 value counts:
청구지수령_K톡
0    2930
1      70
Name: count, dtype: int64

청구지수령_문자메세지 value counts:
청구지수령_문자메세지
0    2688
1     312
Name: count, dtype: int64

청구지수령_미수령 value counts:
청구지수령_미수령
0    2947
1      53
Name: count, dtype: int64

청구지수령_우편 value counts:
청구지수령_우편
1    1572
0    1428
Name: count, dtype: int64

청구지수령_이메일 value counts:
청구지수령_이메일
0    2007
1     993
Name: count, dtype: int64


## OS구분 코드
원핫 인코딩 처리리

In [155]:
# Create dummy variables for OS구분코드
os_dummies = pd.get_dummies(df['OS구분코드'], prefix='OS')

# Drop original column and concatenate dummy variables
df = pd.concat([df.drop('OS구분코드', axis=1), os_dummies], axis=1)

# Fill NA values with 0
os_cols = [col for col in df.columns if col.startswith('OS_')]
df[os_cols] = df[os_cols].fillna(0)

# Convert to integer type
df[os_cols] = df[os_cols].astype(int)

# Print value counts for verification
for col in os_cols:
    print(f"\n{col} value counts:")
    print(df[col].value_counts())

KeyError: 'OS구분코드'

In [148]:
## '할인건수_R3M', '할인건수_B0M'

In [149]:
check_unique_values(df, '할인건수_R3M')
check_unique_values(df, '할인건수_B0M')

=== Analysis for column: 할인건수_R3M ===
Unique values and their counts:
1회 이상: 2758
10회 이상: 163
20회 이상: 67
30회 이상: 12
Total unique values: 4
NULL values: 0

=== Analysis for column: 할인건수_B0M ===
Unique values and their counts:
1회 이상: 2983
10회 이상: 17
Total unique values: 2
NULL values: 0



In [ ]:
##  '인입횟수_ARS_R6M', '이용메뉴건수_ARS_R6M', '방문횟수_PC_R6M', '방문일수_PC_R6M', '방문횟수_앱_R6M'

In [150]:
check_unique_values(df, '인입횟수_ARS_R6M')
check_unique_values(df, '이용메뉴건수_ARS_R6M')
check_unique_values(df, '방문횟수_PC_R6M')
check_unique_values(df, '방문일수_PC_R6M')
check_unique_values(df, '방문횟수_앱_R6M')
check_unique_values(df, '캠페인접촉건수_R12M')
check_unique_values(df, '캠페인접촉일수_R12M')


=== Analysis for column: 인입횟수_ARS_R6M ===
Unique values and their counts:
1회 이상: 2929
10회 이상: 71
Total unique values: 2
NULL values: 0

=== Analysis for column: 이용메뉴건수_ARS_R6M ===
Unique values and their counts:
1회 이상: 2919
10회 이상: 81
Total unique values: 2
NULL values: 0

=== Analysis for column: 방문횟수_PC_R6M ===
Unique values and their counts:
1회 이상: 2833
10회 이상: 84
20회 이상: 68
30회 이상: 15
Total unique values: 4
NULL values: 0

=== Analysis for column: 방문일수_PC_R6M ===
Unique values and their counts:
1회 이상: 2846
10회 이상: 111
20회 이상: 43
Total unique values: 3
NULL values: 0

=== Analysis for column: 방문횟수_앱_R6M ===
Unique values and their counts:
1회 이상: 2690
10회 이상: 93
30회 이상: 62
20회 이상: 59
40회 이상: 42
60회 이상: 26
50회 이상: 19
70회 이상: 8
80회 이상: 1
Total unique values: 9
NULL values: 0



In [153]:
check_unique_values(df, 'OS구분코드')


=== Analysis for column: OS구분코드 ===
Unique values and their counts:
Android: 697
IOS: 232
Total unique values: 2
NULL values: 2071
NULL percentage: 69.03%

